<a href="https://colab.research.google.com/github/phenixua/HW_10/blob/main/HW_10_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.color import gray2rgb
from tensorflow.keras.callbacks import EarlyStopping

# Завантаження даних Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Зміна розміру зображень на 32x32 та перетворення в RGB
x_train_resized = np.array([gray2rgb(np.resize(image, (32, 32))) for image in x_train])
x_test_resized = np.array([gray2rgb(np.resize(image, (32, 32))) for image in x_test])

# Перетворення міток у one-hot вектори
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Розділення тренувального набору на тренувальний та валідаційний
x_train, x_val, y_train, y_val = train_test_split(x_train_resized, y_train, test_size=0.1, random_state=42)

# Завантаження архітектури VGG16 без верхніх повністю зв'язаних шарів
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Додавання нових шарів
model = Sequential([
    vgg_base,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Зняття замороження з ваг моделі VGG16
vgg_base.trainable = True

# Компіляція моделі
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Виведення інформації про модель
model.summary()

# Додавання ранньої зупинки
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Навчання моделі
history = model.fit(x_train, y_train, epochs=30, batch_size=128, validation_data=(x_val, y_val), callbacks=[early_stopping])

# Оцінка точності на тестових даних
test_loss, test_acc = model.evaluate(x_test_resized, y_test)
print(f'Test accuracy: {test_acc}')


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_4 (Flatten)         (None, 512)               0         
                                                                 
 dense_12 (Dense)            (None, 512)               262656    
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                                 
 dense_14 (Dense)            (None, 10)               

Точність отриманої згорткової мережі з точністю багатошарової мережі майже однакові.
У випадку порівняння між цими двома моделями, багатошарова нейронна мережа має дещо кращу точність, але згорткова мережа може бути більш ефективною з точки зору ресурсів та складності моделі.